In [30]:
import pandas as pd
import numpy as np
# !pip install kmodes

# import kmodes as km

In [31]:
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')

In [32]:
rowdata = pd.read_csv('../data/Wayfair-Babson_HackathonData 2019.csv')

In [33]:
data = rowdata.copy()

## Feature Engineering on State
* Segment the states based on their area 
* State separated by Eastern, Mid_West, West_Coast,South_West, Other_usa,  non_usa

In [35]:


# data['non_usa'] = data.State.apply(lambda x:1 if x in non_usa else 0)
# data  = data.query('non_usa == 0')
# wf.head()


# drop null values in State
# data = data[data.State.notnull()]
# wf.shape

non_usa =  ['AB', 'ACT', 'BC', 'CLA', 'DON', 'DUB', 'MB', 'NAN', 'NB', 'NL', 'NS',
            'NSW', 'NT', 'NU', 'ON', 'PE', 'QC', 'QLD', 'SA', 'SK', 'TAS','VIC','WAT','YT']

Eastern =  ['AL','CT','DC','DE','FL','GA','IL','KY','MA','MD','ME',
        'MI','MS','NC','NH','NJ','NY','OH','PA','RI','SC','TN','VA','VT','WV']

West_Coast = ['WA','OR','CA']

Mid_West = ['AR','CO','IA','ID','IN','KS','LA','MN','MO','MT','ND','NE','NV','OK','SD','UT','WI','WY']

South_West = ['AZ','NM','TX']
Other =  ['AK','HI','PR','VI']

data['State'] = data.State.apply(lambda x:'eastern' if x in Eastern else x)
data['State'] = data.State.apply(lambda x:'west_coast' if x in West_Coast else x)
data['State']  = data.State.apply(lambda x:'mid_west' if x in Mid_West else x)
data['State'] = data.State.apply(lambda x:'south_west' if x in South_West else x)
data['State'] = data.State.apply(lambda x:'other_usa' if x in Other else x)
data['State'] = data.State.apply(lambda x:'non_usa' if x in non_usa else x)

data.shape

(992977, 18)

In [36]:
data.head()

,VisitDate,UniqueVisitID,VisitorGroup,PlatformUsed,VisitSource,BrowserName,OSName,State,Gender,IncomeRange,ViewedProductInVisit,ViewedSaleInVisit,TotalPageViews,PlacedSearch,SecondsOnSite,ClickedBanner,AddedToBasket,Purchased
0,7/1/2017,2014237601010030,Prior Visitor with Purchase,Web,ThirdPartyAds,Chrome,Mac OS X,eastern,Female,100to250k,0,0,1,0,0,0,0,0
1,7/1/2017,2909699474086000,New Visitor,Phone,WebSearch,Safari,iOS,NaN,NaN,NaN,1,0,9,0,230,0,0,0
2,7/1/2017,2987400996435030,New Visitor,Web,WebSearch,Chrome,Windows,NaN,NaN,NaN,0,0,28,0,10412,0,0,0
3,7/1/2017,6875972920873110,Prior Visitor with Purchase,Phone,EmailFromWayfair,Safari,iOS,west_coast,Female,100to250k,1,1,8,1,430,0,0,0
4,7/1/2017,11534913004844900,New Visitor,Phone,ThirdPartyAds,Safari,iOS,NaN,NaN,NaN,1,1,9,0,1158,0,0,0


(992977, 18)

## Feature Generation b_effective
* Made an assumption: IF a customer clicked the banner and added something into the basket/cart or if the consumer clicked the banner and purchased something, then the banner is effective in this sepecific visiting recoding.

In [39]:
data = data.reset_index()

%%time
data['b_effective'] = 0
for i in range(len(data.index)):
    
    if data['ClickedBanner'][i] == 1 and (data['AddedToBasket'][i]==1 or data['Purchased'][i]==1):
        data['b_effective'][i]=1
    

CPU times: user 9min 32s, sys: 4.57 s, total: 9min 36s
Wall time: 9min 47s


In [42]:
# data=data.drop(['AddedToBasket','ClickedBanner','Purchased'],axis =1)
data=data.drop(['AddedToBasket','Purchased'],axis =1)

KeyError: "['AddedToBasket' 'Purchased'] not found in axis"

In [45]:
data.query('b_effective ==1')

,VisitDate,UniqueVisitID,VisitorGroup,PlatformUsed,VisitSource,BrowserName,OSName,State,Gender,IncomeRange,ViewedProductInVisit,ViewedSaleInVisit,TotalPageViews,PlacedSearch,SecondsOnSite,ClickedBanner,b_effective
102,7/1/2017,450095063575678000,Prior Visitor with Purchase,Web,EmailFromWayfair,Safari,iOS,mid_west,Female,Under50k,1,1,14,1,623,1,1
113,7/1/2017,497516998518176000,Prior Visitor with Purchase,Web,EmailFromWayfair,Chrome,Windows,eastern,Female,50to100k,1,1,29,1,1119,1,1
154,7/1/2017,756088562101329000,Prior Visitor with Purchase,Phone,EmailFromWayfair,Safari,iOS,NaN,Unknown,NaN,1,1,19,0,542,1,1
237,7/1/2017,1072403380447570000,Prior Visitor with Purchase,Phone,EmailFromWayfair,Chrome,Android,eastern,Male,Under50k,1,1,24,0,725,1,1
245,7/1/2017,1128125907966530000,Prior Visitor with Purchase,Web,EmailFromWayfair,Chrome,Windows,NaN,Female,NaN,1,1,75,1,2601,1,1
453,7/1/2017,2164798576215250000,New Visitor,Web,ThirdPartyAds,Chrome,Windows,NaN,Female,NaN,1,0,50,1,1919,1,1
458,7/1/2017,2183372437928820000,Prior Visitor with Purchase,Web,WebSearch,Safari,Mac OS X,eastern,Female,50to100k,1,1,47,0,9876,1,1
508,7/1/2017,2413982373191940000,Prior Visitor with Purchase,Phone,EmailFromWayfair,Safari,iOS,NaN,Unknown,NaN,1,1,13,0,228,1,1
737,7/1/2017,3544042677540020000,New Visitor,Web,ThirdPartyAds,Chrome,Windows,eastern,Unknown,50to100k,1,0,97,1,4123,1,1
853,7/1/2017,4182558184147070000,Prior Visitor with Purchase,Phone,EmailFromWayfair,Chrome,Android,NaN,Unknown,NaN,1,1,116,1,4805,1,1


In [44]:

data = data.drop ('index', axis=1)
data.shape

(992977, 17)

In [46]:
data.to_csv('../data/data_categotical.csv', index = False)

# Data prepocessing for GBDT

In [47]:
data1 = data.copy()

In [48]:
data1

,VisitDate,UniqueVisitID,VisitorGroup,PlatformUsed,VisitSource,BrowserName,OSName,State,Gender,IncomeRange,ViewedProductInVisit,ViewedSaleInVisit,TotalPageViews,PlacedSearch,SecondsOnSite,ClickedBanner,b_effective
0,7/1/2017,2014237601010030,Prior Visitor with Purchase,Web,ThirdPartyAds,Chrome,Mac OS X,eastern,Female,100to250k,0,0,1,0,0,0,0
1,7/1/2017,2909699474086000,New Visitor,Phone,WebSearch,Safari,iOS,NaN,NaN,NaN,1,0,9,0,230,0,0
2,7/1/2017,2987400996435030,New Visitor,Web,WebSearch,Chrome,Windows,NaN,NaN,NaN,0,0,28,0,10412,0,0
3,7/1/2017,6875972920873110,Prior Visitor with Purchase,Phone,EmailFromWayfair,Safari,iOS,west_coast,Female,100to250k,1,1,8,1,430,0,0
4,7/1/2017,11534913004844900,New Visitor,Phone,ThirdPartyAds,Safari,iOS,NaN,NaN,NaN,1,1,9,0,1158,0,0
5,7/1/2017,14652409111989100,Prior Visitor with Purchase,Phone,ThirdPartyAds,Safari,iOS,eastern,Unknown,NaN,0,0,3,1,30,0,0
6,7/1/2017,14963817946934700,New Visitor,Web,WebSearch,Chrome,Chrome OS,NaN,NaN,NaN,1,0,5,0,284,0,0
7,7/1/2017,25989738567729000,Prior Visitor with Purchase,Phone,ThirdPartyAds,Safari,iOS,NaN,Unknown,NaN,1,0,21,1,1290,0,0
8,7/1/2017,29576725167987100,New Visitor,Phone,ThirdPartyAds,Safari,iOS,NaN,NaN,NaN,1,0,5,1,139,0,0
9,7/1/2017,33737592140659800,Prior Visitor with Purchase,Phone,WebSearch,Safari,iOS,NaN,Female,NaN,0,0,4,0,136,0,0


NameError: name 'data1' is not defined